In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
import glob
file_list = glob.glob("data/*.tsv")



In [5]:
def map_genome(base_pair):
            if base_pair == 'A':
                return 0
            elif base_pair == 'T':
                return 1
            elif base_pair == 'C':
                return 2
            elif base_pair == 'G':
                return 3


            # Go from file name to list of length 3994, each entry w/ 330 base pair sequence
def file_to_matrix(file_name):
    genome_data = pd.read_csv('data/SRR3721963_wgmlst_sig.tsv', sep='\t',
                              names = ['foo', 'bar', 'genome'], skiprows=3)
    genome = genome_data['genome']
    #coded_seq = np.zeros(len(
    coded_seq = []

    for seq in genome:
        mapped_seq = list(map(map_genome, list(seq)))
        coded_seq.append(mapped_seq)

    return coded_seq



In [ ]:
file_

In [ ]:
data_matrices = list(map(file_to_matrix, file_list))
# 241 samples, each with 1,318,020 base pairs


In [ ]:
print(data_matrices[0])


In [4]:
clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train, y_train)

In [ ]:
# Test and check accuracy
print(clf.score(X_test, y_test)